In [1]:
# General imports

import numpy as np
import pandas as pd
import datetime as dt
import os, gc, math, warnings
import psycopg2

from sklearn.preprocessing import LabelEncoder

In [2]:
########################### Helpers
#################################################################################
## -------------------
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
## -------------------


In [3]:
#Variables
path = os.path.join(os.path.expanduser('~'), 'Documents', 'github.passwords', 'energy.predictor.2020.config')
print (path)

myfile = open(path,"rt")
config= myfile.read()

/home/douglas/Documents/github.passwords/energy.predictor.2020.config


In [4]:
#submission

#  Read PostgreSQL into dataframe
#connection
connection = psycopg2.connect(config)

cursor = connection.cursor()

sql = '''
        SELECT
            row_id,
            meter_reading
        FROM
            submission
       '''

submission = pd.read_sql_query(sql,connection) 

if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [5]:
#data check
print("Main:",list(submission),submission.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype
---  ------         -----
 0   row_id         int64
 1   meter_reading  int64
dtypes: int64(2)
memory usage: 636.3 MB
Main: ['row_id', 'meter_reading'] None


In [6]:
#count lines

#  Read PostgreSQL into dataframe
#connection
connection = psycopg2.connect(config)

cursor = connection.cursor()

sql = '''
        SELECT
            COUNT(t.timestamp)
        FROM
            weather_train as w
        INNER JOIN building_metadata as bm
            ON (bm.site_id = w.site_id)
        INNER JOIN train as t
            ON (t.timestamp = w.timestamp) AND (t.building_id = bm.building_id)
        WHERE
                         w.timestamp IS NOT NULL 
             AND w.air_temperature IS NOT NULL 
             AND w.dew_temperature IS NOT NULL
             AND w.sea_level_pressure IS NOT NULL 
             AND w.wind_direction IS NOT NULL 
             AND w.wind_speed IS NOT NULL
       '''

number_lines = pd.read_sql_query(sql,connection) 

if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [7]:
#number_lines

In [10]:
#SQL JOIN Most likely not working because gaps in the weather timestamps, so meter reading timestamps being eliminated


#  Read PostgreSQL into dataframe
#connection
connection = psycopg2.connect(config)

cursor = connection.cursor()

sql = '''
        SELECT

            t.meter,
            t.timestamp,
            t.meter_reading,
            bm.primary_use
            
        FROM
            train as t
        LEFT JOIN building_metadata as bm
            ON (bm.building_id = t.building_id)
        FULL JOIN weather_train as w
            ON (t.timestamp=w.timestamp)
        WHERE 
            w.site_id = bm.site_id

       '''

train_df = pd.read_sql_query(sql,connection) 

if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [11]:
#data check
print("Main:",list(train_df),train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20125605 entries, 0 to 20125604
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   meter          int64         
 1   timestamp      datetime64[ns]
 2   meter_reading  float64       
 3   primary_use    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 614.2+ MB
Main: ['meter', 'timestamp', 'meter_reading', 'primary_use'] None


In [36]:
#  Read PostgreSQL into dataframe
#connection
connection = psycopg2.connect(config)

cursor = connection.cursor()

sql = '''
        SELECT
            bm.building_id,
            bm.site_id,
            t.meter,
            t.timestamp,
            bm.primary_use,
            bm.square_feet,
            bm.year_built,
            bm.floor_count,
            w.air_temperature,
            w.cloud_coverage,
            w.dew_temperature,
            w.precip_depth_1_hr,
            w.sea_level_pressure,
            w.wind_direction,
            w.wind_speed       
            
        FROM
            weather_test as w
        INNER JOIN building_metadata as bm
            ON (bm.site_id = w.site_id)
        INNER JOIN test as t
            ON (t.timestamp = w.timestamp) AND (t.building_id = bm.building_id)
       '''

test_df = pd.read_sql_query(sql,connection) 

if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL connection is closed


In [37]:
#data check
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41498571 entries, 0 to 41498570
Data columns (total 15 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int64         
 1   site_id             int64         
 2   meter               int64         
 3   timestamp           datetime64[ns]
 4   primary_use         object        
 5   square_feet         int64         
 6   year_built          float64       
 7   floor_count         float64       
 8   air_temperature     float64       
 9   cloud_coverage      float64       
 10  dew_temperature     float64       
 11  precip_depth_1_hr   float64       
 12  sea_level_pressure  float64       
 13  wind_direction      float64       
 14  wind_speed          float64       
dtypes: datetime64[ns](1), float64(9), int64(4), object(1)
memory usage: 4.6+ GB
Main: ['building_id', 'site_id', 'meter', 'timestamp', 'primary_use', 'square_feet', 'year_built', 'floor_count', 'air_tempe

In [38]:
#preprocess
#timestamp
for df in [train_df,test_df]:
    df['DT_month'] = df['timestamp'].dt.month.astype(np.int8)
    df['DT_week_y'] = df['timestamp'].dt.isocalendar().week.astype(np.int8)
    df['DT_day_y'] = df['timestamp'].dt.dayofyear.astype(np.int8)
    df['DT_hour'] = df['timestamp'].dt.hour.astype(np.int8)
    df['DT_day_w'] = df['timestamp'].dt.dayofweek.astype(np.int8)
    df['DT_day_m'] = df['timestamp'].dt.day.astype(np.int8)
    df['DT_week_m'] = df['timestamp'].dt.day/7
    df['DT_week_m'] = df['DT_week_m'].apply(lambda x: math.ceil(x)).astype(np.int8)    
    

#string to category
le = LabelEncoder()

for df in [train_df,test_df]:
    df['primary_use'] = df['primary_use'].astype('category')
    df['primary_use'] = df['primary_use'].astype(str)
    df['primary_use'] = le.fit_transform(df['primary_use']).astype(np.int8)

#fill empty - floor count NA = 0, year built = -999
    df['floor_count'] = df['floor_count'].fillna(0).astype(np.int8)
    df['year_built'] = df['year_built'].fillna(0).astype(np.int8)

In [39]:
#data check
print("Main:",list(train_df),train_df.info())
print("#" * 20)
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17684654 entries, 0 to 17684653
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int64         
 1   site_id             int64         
 2   meter               int64         
 3   timestamp           datetime64[ns]
 4   meter_reading       float64       
 5   primary_use         int8          
 6   square_feet         int64         
 7   year_built          int8          
 8   floor_count         int8          
 9   air_temperature     float64       
 10  cloud_coverage      float64       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float64       
 13  sea_level_pressure  float64       
 14  wind_direction      float64       
 15  wind_speed          float64       
 16  DT_month            int8          
 17  DT_week_y           int8          
 18  DT_day_y            int8          
 19  DT_hour             int8          
 20  

In [40]:
do_not_convert = ['category','datetime64[ns]','object']
for df in [train_df, test_df]:
    original = df.copy()
    df = reduce_mem_usage(df)

    for col in list(df):
        if df[col].dtype.name not in do_not_convert:
            if (df[col]-original[col]).sum()!=0:
                df[col] = original[col]
                print('Bad transformation', col)

Mem. usage decreased to 742.08 Mb (61.4% reduction)
Bad transformation meter_reading
Bad transformation air_temperature
Bad transformation dew_temperature
Bad transformation sea_level_pressure
Bad transformation wind_speed
Mem. usage decreased to 1583.04 Mb (62.3% reduction)
Bad transformation air_temperature
Bad transformation dew_temperature
Bad transformation sea_level_pressure
Bad transformation wind_speed


In [41]:
#data check#data check
print("Main:",list(train_df),train_df.info())
print("#" * 20)
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17684654 entries, 0 to 17684653
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int16         
 1   site_id             int8          
 2   meter               int8          
 3   timestamp           datetime64[ns]
 4   meter_reading       float64       
 5   primary_use         int8          
 6   square_feet         int32         
 7   year_built          int8          
 8   floor_count         int8          
 9   air_temperature     float64       
 10  cloud_coverage      float16       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float16       
 13  sea_level_pressure  float64       
 14  wind_direction      float16       
 15  wind_speed          float64       
 16  DT_month            int8          
 17  DT_week_y           int8          
 18  DT_day_y            int8          
 19  DT_hour             int8          
 20  

In [42]:
submission.to_pickle('submission.pkl')
train_df.to_pickle('train_df.pkl')
test_df.to_pickle('test_df.pkl')